In [1]:
import sys
import os

# Agregar la ruta al proyecto para importar core desde security
sys.path.append("C:/Desarrollo/strix-system")

from security.services.user_service import UserService
from security.services.access_control_service import AccessControlService
from core.infraestructure.uow.sqlalchemy_uow import SQLAlchemyUnitOfWork
from sqlalchemy.exc import IntegrityError

def crear_usuario_con_rol_y_permiso(username, email, password, full_name, role_name, permission_name, permission_description=None, role_description=None):
    user_service = UserService()
    access_service = AccessControlService()

    with SQLAlchemyUnitOfWork() as uow:
        # 1. Crear permiso
        permission = access_service.get_or_create_permission(
            name=permission_name,
            description=permission_description,
            uow=uow
        )
        print("✅ Permiso listo:", permission)

        # 2. Crear rol
        role = access_service.get_or_create_role(
            name=role_name,
            description=role_description,
            uow=uow
        )
        print("✅ Rol listo:", role)

        # 3. Asignar permiso al rol
        access_service.assign_permission_to_role(
            role_id=role["id"],
            permission_id=permission["id"],
            uow=uow
        )
        print("✅ Permiso asignado al rol")

        # 4. Crear usuario
        user = user_service.register_user(
            username=username,
            email=email,
            password=password,
            full_name=full_name,
            uow=uow
        )
        if user is None:
            print(f"⚠️ No se creó el usuario '{username}' (puede que ya exista)")
            return None

        print("✅ Usuario creado:", user)

        # 5. Asignar rol al usuario
        access_service.assign_role_to_user(
            user_id=user["id"],
            role_id=role["id"],
            uow=uow
        )
        print("✅ Rol asignado al usuario")

        # Commit final (podría omitirse si cada método hace commit interno)
        uow.commit()

        return {
            "user": user,
            "role": role,
            "permission": permission
        }


In [2]:
crear_usuario_con_rol_y_permiso(
    username="testuser991",
    email="test99@example1.com",
    password="testpass1231",
    full_name="Tester 991",
    role_name="tester_role2",
    permission_name="view_reports2",
    permission_description="Puede ver reportes2",
    role_description="Rol de prueba"
)

✅ Nueva Conexión a PostgreSQL establecida.
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
✅ Permiso listo: {'id': 70, 'name': 'view_reports2', 'description': 'Puede ver reportes2'}
✅ Rol listo: {'id': 69, 'name': 'tester_role2', 'description': 'Rol de prueba'}
✅ Permiso asignado al rol
⚠️ Usuario 'testuser991' ya existe.
⚠️ No se creó el usuario 'testuser991' (puede que ya exista)


In [1]:
import sys
import os

# Agregar la ruta al proyecto para importar core desde security
sys.path.append("C:/Desarrollo/strix-system")

In [3]:
from security.services.user_service import UserService
from security.services.access_control_service import AccessControlService
from core.infraestructure.uow.sqlalchemy_uow import SQLAlchemyUnitOfWork

#from security.data_access.user_repository import UserRepository


In [5]:
user_service = UserService()
access_service = AccessControlService()

with SQLAlchemyUnitOfWork() as uow:
    user_repo = uow.users  # ¡ya está instanciado con sesión!

username = "testuser991"
password = "testpass1231"
email = "test99@example1.com"
full_name = "Tester 991"
role_name = "tester_role2"
permission_name = "view_reports2"

permission = access_service.get_or_create_permission(name=permission_name, description="Puede ver reportes2")
assert permission is not None
print("✅ Permiso obtenido o creado:", permission)

🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
✅ Permiso obtenido o creado: {'id': 70, 'name': 'view_reports2', 'description': 'Puede ver reportes2'}


In [6]:
# 2. Crear el rol
role = access_service.create_role(name=role_name, description="Rol de prueba")
assert role is not None
print("✅ Rol creado:", role)

🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
✅ Rol creado: {'id': 69, 'name': 'tester_role2', 'description': 'Rol de prueba'}


In [7]:
# 3. Asignar permiso al rol
access_service.assign_permission_to_role(role_id=role["id"], permission_id=permission["id"])
print(f"✅ Permiso '{permission_name}' asignado al rol '{role_name}'")

🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
✅ Permiso 'view_reports2' asignado al rol 'tester_role2'


In [8]:
# 4. Crear el usuario
user = user_service.register_user(username=username, email=email, password=password, full_name=full_name)
user


🚨 ENTER: Se abrió una nueva conexión SQLAlchemy
⚠️ Usuario 'testuser991' ya existe.


TypeError: 'NoneType' object is not subscriptable

In [10]:
# 5. Asignar el rol al usuario
access_service.assign_role_to_user(user_id=user["id"], role_id=role["id"])
print(f"✅ Rol '{role_name}' asignado al usuario '{username}'")

TypeError: 'NoneType' object is not subscriptable

In [11]:
# 6. Validar login y recuperar datos del usuario con roles y permisos
result = user_repo.validate_user_credentials(username=username, raw_password=password)
assert result is not None
assert "user" in result
assert "roles" in result
print("✅ Login correcto. Usuario autenticado.")

✅ Login correcto. Usuario autenticado.


In [13]:
roles = result["roles"]
assert any(r["name"] == role_name for r in roles), "El usuario no tiene el rol esperado"
print("✅ El usuario tiene el rol asignado:", role_name)

found_perm = any(p["name"] == permission_name for r in roles for p in r["permissions"])
assert found_perm, "El usuario no tiene el permiso esperado"
print("✅ El usuario tiene el permiso esperado:", permission_name)

print("🎉 TEST COMPLETO: Todo funcionó correctamente")

✅ El usuario tiene el rol asignado: tester_role2
✅ El usuario tiene el permiso esperado: view_reports2
🎉 TEST COMPLETO: Todo funcionó correctamente
